**Name: XXXXXX     Student ID: pxxxxxx**

**Import libraries**

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

**Load dataset**

In [ ]:
df = pd.read_excel('/content/Online Retail.xlsx')

In [ ]:
df.shape

In [ ]:
df.head()

**Data cleaning**

In [ ]:
# Remove data records with negative quantity
df.loc[df['Quantity'] <= 0].shape

In [ ]:
df = df.loc[df['Quantity'] > 0]

In [ ]:
df.shape

In [ ]:
# Number of data records with empty CustomerID
pd.isnull(df['CustomerID']).sum()

In [ ]:
# Remove data records with missing CustomerID
df = df[pd.notnull(df['CustomerID'])]

In [ ]:
df.shape

In [ ]:
df.head()

**Exclude incomplete month**

In [ ]:
print('Date Range: %s ~ %s' % (df['InvoiceDate'].min(), df['InvoiceDate'].max()))

In [ ]:
df.loc[df['InvoiceDate'] >= '2011-12-01'].shape

In [ ]:
# Remove data records which the invoice date is between 2011-12-01 and 2011-12-09
df = df.loc[df['InvoiceDate'] < '2011-12-01']

In [ ]:
df.shape

**Total sales**

In [ ]:
# Create a new feature called Sales = Quantity x UnitPrice
df['Sales'] = df['Quantity'] * df['UnitPrice']

In [ ]:
df.head()

**Per Order Data**

In [ ]:
# For each customer and each invoice number, get the total sales and latest invoice date
orders_df = df.groupby(['CustomerID', 'InvoiceNo']).agg({
    'Sales': sum,
    'InvoiceDate': max
})

In [ ]:
orders_df

**Data analysis**

In [ ]:
def groupby_mean(x):
    return x.mean()

def groupby_count(x):
    return x.count()

def purchase_duration(x):
    return (x.max() - x.min()).days

def avg_frequency(x):
    return (x.max() - x.min()).days/x.count()

groupby_mean.__name__ = 'avg'
groupby_count.__name__ = 'count'
purchase_duration.__name__ = 'purchase_duration'
avg_frequency.__name__ = 'purchase_frequency'

In [ ]:
# For each customer, get the min, max, sum, average, count of Sales, and
# min, max, purchase_duration, purchase_frequency of Invoice Date
summary_df = orders_df.reset_index().groupby('CustomerID').agg({
    'Sales': [min, max, sum, groupby_mean, groupby_count],
    'InvoiceDate': [min, max, purchase_duration, avg_frequency]
})

In [ ]:
summary_df

In [ ]:
summary_df.columns = ['_'.join(col).lower() for col in summary_df.columns]

In [ ]:
summary_df

In [ ]:
# Consider only the customers whose purchase duration is larger than 0 (i.e., those customers who have bought more than once!)
summary_df = summary_df.loc[summary_df['invoicedate_purchase_duration'] > 0]

In [ ]:
ax = summary_df.groupby('sales_count').count()['sales_avg'][:20].plot(
    kind='bar', 
    color='skyblue',
    figsize=(12,7), 
    grid=True
)

ax.set_ylabel('count')

plt.show()

In [ ]:
summary_df['sales_count'].describe()

In [ ]:
summary_df['sales_avg'].describe()

In [ ]:
ax = summary_df['invoicedate_purchase_frequency'].hist(
    bins=20,
    color='skyblue',
    rwidth=0.7,
    figsize=(12,7)
)

ax.set_xlabel('avg. number of days between purchases')
ax.set_ylabel('count')

plt.show()

In [ ]:
summary_df['invoicedate_purchase_frequency'].describe()

In [ ]:
summary_df['invoicedate_purchase_duration'].describe()

**Linear Regression**

In [ ]:
# Predict the CLV of next 3 months
clv_freq = '3M'

In [ ]:
# Set the invoice date to be the last day of every 3 months
data_df = orders_df.reset_index().groupby([
    'CustomerID',
    pd.Grouper(key='InvoiceDate', freq=clv_freq)
]).agg({
    'Sales': [sum, groupby_mean, groupby_count],
})

In [ ]:
data_df.columns = ['_'.join(col).lower() for col in data_df.columns]

In [ ]:
data_df = data_df.reset_index()

In [ ]:
data_df.head(10)

In [ ]:
# Sort invoice date in descending order
date_month_map = {
    str(x)[:10]: 'M_%s' % (i+1) for i, x in enumerate(
        sorted(data_df.reset_index()['InvoiceDate'].unique(), reverse=True)
    )
}

In [ ]:
data_df['M'] = data_df['InvoiceDate'].apply(lambda x: date_month_map[str(x)[:10]])

In [ ]:
date_month_map

In [ ]:
data_df.head(10)

**Building sample set**

In [ ]:
# Create a data matrix excluding M_1 as it will be used as dependent variable
features_df = pd.pivot_table(
    data_df.loc[data_df['M'] != 'M_1'], 
    values=['sales_sum', 'sales_avg', 'sales_count'], 
    columns='M', 
    index='CustomerID'
)

In [ ]:
features_df.columns = ['_'.join(col) for col in features_df.columns]

In [ ]:
features_df.shape

In [ ]:
features_df.head(10)

In [ ]:
# Fill NaN with 0
features_df = features_df.fillna(0)

In [ ]:
features_df.head()

In [ ]:
# Set the most recent 3 months sales as the CLV
response_df = data_df.loc[
    data_df['M'] == 'M_1',
    ['CustomerID', 'sales_sum']
]

In [ ]:
response_df.columns = ['CustomerID', 'CLV_'+clv_freq]

In [ ]:
response_df.shape

In [ ]:
response_df.head(10)

In [ ]:
sample_set_df = features_df.merge(
    response_df, 
    left_index=True, 
    right_on='CustomerID',
    how='left'
)

In [ ]:
sample_set_df.shape

In [ ]:
sample_set_df.head(10)

In [ ]:
sample_set_df = sample_set_df.fillna(0)

In [ ]:
sample_set_df.head()

In [ ]:
sample_set_df['CLV_'+clv_freq].describe()

**Regression model**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Set CLV as dependent variable while the other features (except CustomerID) as independent variables
target_var = 'CLV_'+clv_freq
all_features = [x for x in sample_set_df.columns if x not in ['CustomerID', target_var]]

In [ ]:
# Split data: 70% training & 30% testing
x_train, x_test, y_train, y_test = train_test_split(
    sample_set_df[all_features], 
    sample_set_df[target_var], 
    test_size=0.3,
    random_state = 40
)

In [ ]:
from sklearn.linear_model import LinearRegression

# Try these models as well
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

In [ ]:
reg_fit = LinearRegression()

In [ ]:
reg_fit.fit(x_train, y_train)

In [ ]:
reg_fit.intercept_

In [ ]:
coef = pd.DataFrame(list(zip(all_features, reg_fit.coef_)))
coef.columns = ['feature', 'coef']

coef

**Evaluation**

In [ ]:
from sklearn.metrics import r2_score, median_absolute_error

In [ ]:
train_preds =  reg_fit.predict(x_train)
test_preds = reg_fit.predict(x_test)

**R-squared**

In [ ]:
print('In-Sample R-Squared: %0.4f' % r2_score(y_true=y_train, y_pred=train_preds))
print('Out-of-Sample R-Squared: %0.4f' % r2_score(y_true=y_test, y_pred=test_preds))

**Median Absolute Error**

In [ ]:
print('In-Sample MSE: %0.4f' % median_absolute_error(y_true=y_train, y_pred=train_preds))
print('Out-of-Sample MSE: %0.4f' % median_absolute_error(y_true=y_test, y_pred=test_preds))

**Scatterplot**

In [ ]:
plt.scatter(y_train, train_preds)
plt.plot([0, max(y_train)], [0, max(train_preds)], color='gray', lw=1, linestyle='--')

plt.xlabel('actual')
plt.ylabel('predicted')
plt.title('In-Sample Actual vs. Predicted')
plt.grid()

plt.show()

In [ ]:
plt.scatter(y_test, test_preds)
plt.plot([0, max(y_test)], [0, max(test_preds)], color='gray', lw=1, linestyle='--')

plt.xlabel('actual')
plt.ylabel('predicted')
plt.title('Out-of-Sample Actual vs. Predicted')
plt.grid()

plt.show()

**Question 1: By only consider the average sales (i.e, sales_avg), what is the predicted CLV when sales_avg_M2 = 800, sales_avg_M3 = 300, sales_avg_M4 = 200, sales_avg_M5 = 100?**

Answer 1: 

**Question 2: By only considering the average sales (i.e., sales_avg), please interpret the impact and significance of previous average sales on the CLV.**

Answer 2: 

**Question 3: Suggest ONE sales or marketing strategy the company can take to increase the CLV.**

Answer 3: 

**Question 4: Try predict 6-month CLV instead of 3-month, and discuss which prediction (3 or 6 month CLV) gives you a better out-of-sample performance.**

Answer 4: 